In [1]:
#imports
import numpy as np
import tensorflow as tf
import csv

from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from keras.models import load_model

import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

from keras.models import load_model

# Preprocessing

Cleaning

In [2]:
def clean_borders(dirty):
    image = dirty.copy()
    image[0] = np.zeros(len(image[0]))
    image[-1] = np.zeros(len(image[0]))
    image[:,0] = np.zeros(len(image[:,0]))
    image[:,-1] = np.zeros(len(image[:,0]))
    return image

In [3]:
def filter1x1(dirty):
    image = dirty.copy()
    for v in np.arange(1,29):
        for h in np.arange(1,139):
            PXL = 0
            PXL += (sum(image[v-1,h-1:h+2]) + sum(image[v+1,h-1:h+2]) +             sum(image[v-1:v+2,h-1]) + sum(image[v-1:v+2,h+1] ))
            if PXL < 255 and image[v,h] != 0:
                image[v,h] = 0
    return image

def filter2x2(dirty):
    image = dirty.copy()    
    for v in np.arange(1,28):
        for h in np.arange(1,138):
            PXL = 0
            PXL += (sum(image[v-1,h-1:h+3]) + sum(image[v+2,h-1:h+3]) +             sum(image[v-1:v+3,h-1]) + sum(image[v-1:v+3,h+2]) )
            if PXL < 255 and np.sum(image[v:v+2,h:h+2]) != 0:
                image[v:v+2,h:h+2] = np.zeros(4).reshape((2,2))

    return image

def filter3x3(dirty):
    image = dirty.copy()
    for v in np.arange(1,27):
        for h in np.arange(1,137):
            PXL = 0
            PXL += (sum(image[v-1,h-1:h+4]) + sum(image[v+3,h-1:h+4]) +             sum(image[v-1:v+4,h-1]) + sum(image[v-1:v+4, h+3]))
            if PXL < 360 and np.sum(image[v:v+3,h:h+3]) > 255 and np.sum(image[v:v+3,h:h+3]) < 1720:
                image[v:v+3,h:h+3] = np.zeros(9).reshape((3,3))
    return image

def filter3x1(dirty):
    image = dirty.copy()    
    for v in np.arange(1,27):
        for h in np.arange(1,139):
            PXL = 0
            PXL += (sum(image[v-1:v+4,h-1]) + sum(image[v-1:v+4,h+1]) +            sum(image[v+3,h-1:h+2]) + sum(image[v-1,h-1:h+2]) )
            if PXL < 255 and np.sum(image[v:v+3,h]) != 0:
                image[v:v+3,h] = np.zeros(3).reshape((1,3))
    return image

def filter1x3(dirty):
    image = dirty.copy()
    for v in np.arange(1,29):
        for h in np.arange(1,137):
            PXL = 0
            PXL += (sum(image[v-1,h-1:h+4]) + sum(image[v+1,h-1:h+4]) +             sum(image[v-1:v+2,h-1]) + sum(image[v-1:v+2,h+3]))
            if PXL < 255 and np.sum(image[v,h:h+3]) != 0:
                image[v,h:h+3] = np.zeros(3).reshape((1,3))
    return image

In [4]:
def negative_clean(dirty):
    image = dirty.copy()
    PXL = 100
    for v in np.arange(1,29):
        for h in np.arange(1,139):
            if sum(image[v-1,h-1:h+2]) > PXL and sum(image[v+1,h-1:h+2]) > PXL and             sum(image[v-1:v+2,h-1]) > PXL and sum(image[v-1:v+2,h+1]) > PXL and             image[v,h] < 30 and np.sum(image[v-1:v+2,h-1:h+2]) > 1000:
                image[v,h] = (np.sum(image[v-1:v+2,h-1:h+2]) + 200) / 9
    return image

In [5]:
def remove_image_noise(image):
    clean = clean_borders(image)
    return negative_clean(filter1x1(filter1x3(filter3x1(filter2x2(filter3x3(clean))))))

Image seperation

In [6]:
#all letter list functions 
def letter_list_3(image):
    letter_list = []
    identifier = []
    i = 0        
    while i < (np.shape(image)[1] -1):
        if np.sum(image[:,i:i+2]) > 640:
            identifier.append(i)
    # Until now the separator works the same as in the parent function. The following part is different.
    # The frame that is being recorded is bigger (48x48) if possible, so that combined letters can be separated later on.
        elif sum(identifier) != 0:
            CTR = np.int(np.mean(identifier))
            if CTR < 12:
                x = image[1:-1, 0: 28]
            elif CTR < 20:
                x = image[1:-1, CTR-12: CTR+16]
            elif CTR > 112:
                CTR = 112
                x = image[1:-1, CTR-20: CTR+28]
            else:
                x = image[1:-1, CTR-20: CTR+28]
            letter_list.append(x)
            identifier = []
            i+= 5
        i += 1
    density = []
    
    # The identified letters are being transformed into a single value, stored in the density list.
    # Value is based on the sum of all pixels in the frame.
    # The letter with the highest density is split into two 28x28 frames, this is often the double letter.
    for x in letter_list:
        density = np.append(density, np.sum(x))
        double_letter = letter_list[np.argmax(density)]
        if np.shape(double_letter) == (28,28):
            try:
                double_letter = letter_list[np.argmax(density) + 1]
            except:
                double_letter = letter_list[np.argmax(density) - 1]
    single_letter1 = double_letter[:, 0:28]
    single_letter2 = double_letter[:,20:] 

    # remaining 48x48 frames are being transformed into 28x28 frames.    
    for i, x in enumerate(letter_list):
        if np.shape(x) != (28,28):
            letter_list[i] = x[:,10:38]

    # The separated letters are placed back on the correct position and the old combined frame is removed.
    good_list = []
    for i, x in enumerate(letter_list):
        if i == np.argmax(density):
            good_list.append(single_letter1)
            good_list.append(single_letter2)
        else:
            good_list.append(x)
        
    return good_list
        
def letter_list_2(image):
    letter_list = []
    identifier = []
    i = 0
        
    while i < (np.shape(image)[1] -1):
        if np.sum(image[:,i:i+2]) > 640:
            identifier.append(i)
            
    # Until now the separator works the same as in the parent function. The following part is different.
    # The frame that is being recorded is bigger (48x48) if possible, so that combined letters can be separated later on.
        elif sum(identifier) != 0:
            CTR = np.int(np.mean(identifier))
            if CTR < 20:
                CTR = 20
            if CTR > 112:
                CTR = 112
            x = image[1:-1, CTR-20: CTR+28]
            letter_list.append(x)
            identifier = []
            i+= 5
        i += 1

    new_list = []
    for x in letter_list:
        new_list.append(x[:, 0:28])
        new_list.append(x[:,20:])
    return new_list

def letter_list_1(image):
    letter_list = []
    identifier = []
    i = 0
    
    # While-loop that detects start and beginning of individual letter in a 'Captcha'.
    # A 28x28 frame around the detected center of the letter is stored in the letter_list.
    while i < (np.shape(image)[1] -1):
        if np.sum(image[:,i:i+2]) > 2295:
            identifier.append(i)
        elif sum(identifier) != 0:
            CTR = np.int(np.mean(identifier))
            if CTR < 12:
                CTR = 12
            elif CTR > 124:
                CTR = 124
            x = image[1:-1, CTR-12: CTR+16]
            letter_list.append(x)
            identifier = []
            i+= 5        
        i += 1    
    return letter_list

def letter_list_plus(letter_list):
    for x in np.arange(len(letter_list) - 4):
        density = []
        for x in letter_list:
            density = np.append(density, np.sum(x))

        for i, x in enumerate(letter_list):
            if i == np.argmin(density):
                continue
            letter_list[i] = x
    return letter_list[:4]

In [7]:
#Seperate the image vertically using the letter_list functions
def vertical_separator(image):
    letter_list = []
    identifier = []
    i = 0    
    # While-loop that detects start and beginning of individual letter in a 'Captcha'.
    # A 28x28 frame around the detected center of the letter is stored in the letter_list.
    while i < (np.shape(image)[1] -1):
        if np.sum(image[:,i:i+2]) > 640:
            identifier.append(i)        
        elif sum(identifier) != 0:
            CTR = np.int(np.mean(identifier))
            if CTR < 12:
                CTR = 12
            elif CTR > 124:
                CTR = 124
            x = image[1:-1, CTR-12: CTR+16]
            letter_list.append(x)
            identifier = []
            i+= 5        
        i += 1            
    # While the above function is prone to error, the following if statements check whether the desired results are present. Otherwise different separator is used.  
    if len(letter_list) == 4:
        return letter_list
    
    if len(letter_list) == 3:
        return letter_list_3(image)
    
    if len(letter_list) == 2:
        return letter_list_2(image)

    if len(letter_list) == 1:
        return letter_list_1(image)
    
    if len(letter_list) > 4:
        return letter_list_plus(letter_list)    
    pass

# Validation sets and metrics

In [8]:
def createValidationSets(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=999)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.7777778, random_state=999)
    return x_train, x_test, x_val, y_train, y_test, y_val

In [9]:
def oneHotFitTransform(y_train, y_val, y_test):
    onehot = LabelBinarizer()
    y_train = onehot.fit_transform(y_train)
    y_val   = onehot.transform(y_val)
    y_test   = onehot.transform(y_test)
    return y_train, y_val, y_test

In [10]:
#only used for the accuracy score 
def oneHotInverseTransform(y_pred):
    return onehot.inverse_transform(y_pred)

# Feature engineering 

In [11]:
#reshape 
def reshape_28(array):
    new = []
    for i, x in enumerate(array):
        
        new.append(np.reshape(x,(28,28)))
    return np.array(new)

In [12]:
#crop image
def cropped_image(image, filter_size):
    image_size = np.shape(image)[0]
    cropped_image = []
    # Vertical movement of the filter
    for v in np.arange(0,image_size, filter_size):
        # Horizontal movement of the filter
        for x in np.arange(0,image_size, filter_size):
            cropped_image.append(np.mean(image[v:v+filter_size,x: x + filter_size]))
    return cropped_image

In [13]:
# #get image width old
# def width(image):
#     width_list = []
#     for i, x in enumerate(np.arange(0,np.shape(image)[0] -1)):
#         if sum(image[:,x]) > 300:
#             width_list.append(i)
#             break
#     for i, x in enumerate(np.arange(np.shape(image)[0] -1, 0, -1)):
#         if sum(image[:,x]) > 300:
#             width_list.append(np.shape(image)[0] - i)
#             break
#     return width_list[1] - width_list[0] 

#new
def width(image):
    width_list = []
    for i, x in enumerate(np.arange(0,np.shape(image)[0] -1)):
        if sum(image[:,x]) > 300:
            width_list.append(i)
            break
    for i, x in enumerate(np.arange(np.shape(image)[0] -1, 0, -1)):
        if sum(image[:,x]) > 300:
            width_list.append(np.shape(image)[0] - i)
            break
    if len(width_list) != 2:
        return 20
    return width_list[1] - width_list[0]

In [14]:
# #get image heigth old
# def heigth(image):
#     heigth_list = []
#     for i, x in enumerate(np.arange(0,np.shape(image)[0] -1)):
#         if sum(image[x,:]) > 300:
#             heigth_list.append(i)
#             break
#     for i, x in enumerate(np.arange(np.shape(image)[0] -1, 0, -1)):
#         if sum(image[x,:]) > 300:
#             heigth_list.append(np.shape(image)[0] - i)
#             break
#     return heigth_list[1] - heigth_list[0]   

#new
def heigth(image):
    heigth_list = []
    for i, x in enumerate(np.arange(0,np.shape(image)[0] -1)):
        if sum(image[x,:]) > 300:
            heigth_list.append(i)
            break
    for i, x in enumerate(np.arange(np.shape(image)[0] -1, 0, -1)):
        if sum(image[x,:]) > 300:
            heigth_list.append(np.shape(image)[0] - i)
            break
    if len(heigth_list) != 2:
        return 20
    return heigth_list[1] - heigth_list[0]


In [15]:
#get density
def density(image):
    return  np.sum(image) / (255 * 28 *28)

In [16]:
#get edge count
def edge_count(image):
    flat = image.flatten()
    safety_margin = int(0.05*255)
    edge_count = 0
    for x in flat:
        if x > 0 + safety_margin and x < 255 - safety_margin:
            edge_count += 1
    return edge_count

# Scaler

In [17]:
def fitTransformUsingScaler(data):
    scaler = MaxAbsScaler()
    return scaler.fit_transform(data)

In [18]:
#Scale for CNN model
def scale(X):
    scale = X.astype(dtype = 'float64')
    for i, x in enumerate(scale):
        scale[i] = x / 255.
    return scale

# Train NN model

In [19]:
def createKerasModel():
    # Hyperparameters:
    hidden_nodes = 1024 #?
    output_size = 26
    layer = 'selu'
    kernel_init = 'lecun_normal'
    NUM_EPOCHS = 100 
    eta = 0.00005
    regularizer = tf.keras.regularizers.l2(0.01)
    nadam = tf.keras.optimizers.Nadam(learning_rate=eta, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
    adam = tf.keras.optimizers.Adam(
        learning_rate= eta,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name="Adam",
    )

    model = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(1024, input_dim=269, activation=layer, kernel_initializer = kernel_init, use_bias = False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(512, activation=layer, kernel_initializer = kernel_init, use_bias = False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(512, activation=layer, kernel_initializer = kernel_init, use_bias = False), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(512, activation=layer, kernel_initializer = kernel_init, use_bias = False),
        tf.keras.layers.Dense(output_size, activation = 'softmax')
    ])
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics = ['accuracy'])
    return model

# Train CNN model

In [20]:
def train_CNN_model():
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation="relu", kernel_initializer='lecun_normal'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu", kernel_initializer='lecun_normal'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(26, activation="softmax"),
        ]
    )

    optimizer = tensorflow.keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name="Adam",)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['accuracy'])
    return model

# Predict

In [21]:
#Get the top scores from the predictions and replace the lowest score 
#from all letters with the next best performing predictions
def getTopScores(all_letter_scores, all_letters):
    secondUsedIndex = []
    thirdUsedIndex = []
    fourthUsedIndex = []
    
    resultList = []
    firstScores = []
    secondScores = []
    thirdScores = []
    fourthScores = []
    fifthScores = []
    
    firstPredictions = []
    secondPredictions = []
    thirdPredictions = []
    fourthPredictions = []
    fifthPredictions = []
    
    index = 0
    #add all predictions to a list based on their score
    for scores in all_letter_scores: 
        firstScores.append(scores[0]) 
        secondScores.append(scores[1])
        thirdScores.append(scores[2])
        fourthScores.append(scores[3])
        fifthScores.append(scores[4])
        
        firstPredictions.append(all_letters[index][0]) 
        secondPredictions.append(all_letters[index][1])
        thirdPredictions.append(all_letters[index][2])
        fourthPredictions.append(all_letters[index][3])
        fifthPredictions.append(all_letters[index][4])
        
        index +=1    
    scoreList = [[]]*5
    
    #update the score_list with the highest probability for the 4 letters
    for i in range(5):       
        completeStr = ""    
        indexLowest = firstScores.index(min(firstScores))
        for strScore in firstPredictions:
            #for the sake of reference add to new object
            completeStr += str(strScore).zfill(2)
        scoreList[i] = completeStr   
        
        #if the index is not used before get value from second position otherwise from next
        if (indexLowest not in secondUsedIndex):
            secondUsedIndex.append(indexLowest)
            firstScores[indexLowest] = secondScores[indexLowest]
            firstPredictions[indexLowest] = secondPredictions[indexLowest]
        elif (indexLowest not in thirdUsedIndex):
            thirdUsedIndex.append(indexLowest)
            firstScores[indexLowest] = thirdScores[indexLowest]
            firstPredictions[indexLowest] = thirdPredictions[indexLowest]
        elif (indexLowest not in fourthUsedIndex):
            fourthUsedIndex.append(indexLowest)
            firstScores[indexLowest] = fourthScores[indexLowest]
            firstPredictions[indexLowest] = fourthPredictions[indexLowest]
        else: 
            firstScores[indexLowest] = fifthScores[indexLowest]
            firstPredictions[indexLowest] = fifthPredictions[indexLowest]
    
    return scoreList 

In [29]:

def predictImages(images, model):
    output_list = []
    output_list.append(model.predict(images))

    all_letter_scores = []
    all_letters = []
    letterIndex = range(1, 27)
    #loop over the predictions
    for pred in output_list:
        letterScoreList = []
        letterList = []
        
        #get the top 5 predictions
        score = sorted(zip(pred, letterIndex), reverse=True)[:5]
        letterScoreList.append(score[0][0])
        letterList.append(score[0][1])

        letterScoreList.append(score[1][0])
        letterList.append(score[1][1])

        letterScoreList.append(score[2][0])
        letterList.append(score[2][1])

        letterScoreList.append(score[3][0])
        letterList.append(score[3][1])

        letterScoreList.append(score[4][0])
        letterList.append(score[4][1])

        all_letter_scores.append(letterScoreList)
        all_letters.append(letterList)
    #get the top 5 digit predictions
    return getTopScores(all_letter_scores, all_letters)


In [23]:
letterIndex = range(0, 26)
print(len(letterIndex))

26


# Output

In [32]:
#write prediction to file
def toPredictionFile(all_scores):
    with open('predictionCNN.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(all_scores)

# Load data

In [25]:
#load data
import numpy as np
trainData = np.load("training-dataset.npz")
testData = np.load("test-dataset.npy")

#loadData
x = trainData['x']
y = trainData['y']


# NN predictions 

In [26]:


# #Scale data
# x = fitTransformUsingScaler(x)

# #Create validation sets
#x_train, x_test, x_val, y_train, y_test, y_val = createValidationSets(x, y)

# #OneHot fit and transform
# y_train, y_val, _ = oneHotFitTransform(y_train, y_val, _)

#Create keras NN model
# model = createKerasModel()

# #fit the model with on the training data
# model.fit(x_train,y_train, epochs=100, validation_data = (x_val, y_val), verbose=1)




# #TODO remove
# #for now use pre-trained model
# model = c('DNN_93%.tf')


# # #Clean the images
# cleaned_images = []
# for x in testData[0:2]:
#     clean_image = remove_image_noise(x)
#     cleaned_images.append(vertical_separator(clean_image))

# # predict
# all_scores = []
# for images in cleaned_images:    
#     all_scores.append(predictImages(images, model))

# #export to file
# toPredictionFile(all_scores)

# CNN predictions

In [33]:
#Create CNN model
model = train_CNN_model()
#fit CNN model


#reshape for CNN 
# x = scale(x)
# x = x.reshape((124800,28,28))

#Create validation sets again after reshaping
# x_train, x_test, x_val, y_train, y_test, y_val = createValidationSets(x, y)


#shape train sets for CNN model
# x_train = np.expand_dims(x_train, -1 )
# x_val = np.expand_dims(x_val, -1 )
# x_test = np.expand_dims(x_test, -1)

# #OneHot fit and transform
# y_train, y_val, y_test = oneHotFitTransform(y_train, y_val, y_test)



#model.fit(x_train, y_train,  epochs=25, validation_data=(x_val, y_val), verbose=1)

#todo for now use pre-trained model
model = keras.models.load_model("cnn_model.h5")

# #Clean the images
cleaned_images = []
all_scores = []
#TODO remove [0:2]
for x in testData:
    clean_image = remove_image_noise(x)
    seperated = vertical_separator(clean_image)
    for img in seperated:
        cleaned_images.append(img)
#     print(np.array(cleaned_images[0]).shape)
#     images.append(cleaned_images[0])
#     images = np.array(cleaned_images[0])
#     images = np.expand_dims(images, -1 )
    
images = np.array(cleaned_images)   
images = np.expand_dims(images, -1 )
#all_scores.append(predictImages(images, model))



#not using the function "predictImages()" because this model need a slightly different approach 
output_list = []
output_list.append(model.predict(images))

all_letter_scores = []
all_letters = []
letterIndex = range(1, 27)
#loop over the predictions
for pred in output_list[0]:
    letterScoreList = []
    letterList = []

    #get the top 5 predictions
    score = sorted(zip(pred, letterIndex), reverse=True)[:5]
    letterScoreList.append(score[0][0])
    letterList.append(score[0][1])

    letterScoreList.append(score[1][0])
    letterList.append(score[1][1])

    letterScoreList.append(score[2][0])
    letterList.append(score[2][1])

    letterScoreList.append(score[3][0])
    letterList.append(score[3][1])

    letterScoreList.append(score[4][0])
    letterList.append(score[4][1])

    all_letter_scores.append(letterScoreList)
    all_letters.append(letterList)



resultList = []
firstScores = []
secondScores = []
thirdScores = []
fourthScores = []
fifthScores = []

firstPredictions = []
secondPredictions = []
thirdPredictions = []
fourthPredictions = []
fifthPredictions = []

index = 0
#add all predictions to a list based on their score
for scores in all_letter_scores:
    firstScores.append(scores[0]) 
    secondScores.append(scores[1])
    thirdScores.append(scores[2])
    fourthScores.append(scores[3])
    fifthScores.append(scores[4])

    firstPredictions.append(all_letters[index][0]) 
    secondPredictions.append(all_letters[index][1])
    thirdPredictions.append(all_letters[index][2])
    fourthPredictions.append(all_letters[index][3])
    fifthPredictions.append(all_letters[index][4])

    index +=1    


#split into list of all scores per letter
firstScores_list = [firstScores[x:x+4] for x in range(0, len(firstScores), 4)]
secondScores_list = [secondScores[x:x+4] for x in range(0, len(secondScores), 4)]
thirdScores_list = [thirdScores[x:x+4] for x in range(0, len(thirdScores), 4)]
fourthScores_list = [fourthScores[x:x+4] for x in range(0, len(fourthScores), 4)]
fifthScores_list = [fifthScores[x:x+4] for x in range(0, len(fifthScores), 4)]

#split into list of all predictions per letter
firstPredictions_list = [firstPredictions[x:x+4] for x in range(0, len(firstPredictions), 4)]
secondPredictions_list = [secondPredictions[x:x+4] for x in range(0, len(secondPredictions), 4)]
thirdPredictions_list = [thirdPredictions[x:x+4] for x in range(0, len(thirdPredictions), 4)]
fourthPredictions_list = [fourthPredictions[x:x+4] for x in range(0, len(fourthPredictions), 4)]
fifthPredictions_list = [fifthPredictions[x:x+4] for x in range(0, len(fifthPredictions), 4)]



list_counter = 0
score_list_collection = []
#update the score_list with the highest probability for the 4 letters
for fromFirstScores in firstScores_list:
    secondUsedIndex = []
    thirdUsedIndex = []
    fourthUsedIndex = []
    scoreList = [[]]*5
    for i in range(5):       
        completeStr = ""  
       

        indexLowest = fromFirstScores.index(min(fromFirstScores))
        for strScore in firstPredictions_list[list_counter]:
            #for the sake of reference add to new object
            completeStr += str(strScore).zfill(2)

        scoreList[i] = completeStr   

        #if the index is not used before get value from second position otherwise from next
        if (indexLowest not in secondUsedIndex):
            secondUsedIndex.append(indexLowest)
            fromFirstScores[indexLowest] = secondScores_list[list_counter][indexLowest]
            firstPredictions_list[list_counter][indexLowest] = secondPredictions_list[list_counter][indexLowest]
        elif (indexLowest not in thirdUsedIndex):
            thirdUsedIndex.append(indexLowest)
            fromFirstScores[indexLowest] = thirdScores_list[list_counter][indexLowest]
            firstPredictions_list[list_counter][indexLowest] = thirdPredictions_list[list_counter][indexLowest]
        elif (indexLowest not in fourthUsedIndex):
            fourthUsedIndex.append(indexLowest)
            fromFirstScores[indexLowest] = fourthScores_list[list_counter][indexLowest]
            firstPredictions_list[list_counter][indexLowest] = fourthPredictions_list[list_counter][indexLowest]
        else: 
            fromFirstScores[indexLowest] = fifthScores[indexLowest]
            firstPredictions[indexLowest] = fifthPredictions[indexLowest]
    score_list_collection.append(scoreList)
    list_counter +=1
    
all_scores = score_list_collection


# #export to file
toPredictionFile(all_scores)

In [28]:
from datetime import datetime
#please note this started at 22:40
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 14:53:52
